In [1]:
import pandas as pd
from pandas import read_csv
import numpy as np

# Preparing Training Set

In [2]:
#This block should be completed by the user
name_dic={\
          'PANC1_proliferating_A3':'9191',\
          'PANC1_proliferating_B1':'9197',\
         }
[print(key,value) for key, value in name_dic.items()]

PANC1_proliferating_A3 9191
PANC1_proliferating_B1 9197


[None, None]

In [3]:
#Read in all the data
for key in name_dic:    
    dir_name=str(name_dic[key]) #The data files for each experiment, i.e. key, is in a folder named name_dic[key].
    
    temp_var_name='resp_'+dir_name
    temp_dataframe=read_csv(dir_name+"/responses.csv",header=0)
    exec(temp_var_name + " = temp_dataframe") #Assign the contents of responses.csv to a file prefixed with "resp_"
    
    temp_var_name='meta_'+dir_name
    temp_dataframe=read_csv(dir_name+"/metadata.csv",header=0)
    #temp_dataframe['LogRowIC50']=np.log10(temp_dataframe['RowIC50']*(10**-6))
    #temp_dataframe['LogColIC50']=np.log10(temp_dataframe['ColIC50']*(10**-6))
    exec(temp_var_name + " = temp_dataframe")
    
    temp_var_name='calc_'+dir_name
    temp_dataframe=read_csv(dir_name+"/calc.csv",header=0)
    exec(temp_var_name + " = temp_dataframe")

In [4]:
#The function "data" takes the experiment name, e.g. PANC1_confluent_A1,
#and the name of specific data file, e.g. resp, meta, or calc, and returns the dataframe.
def data(experiment,type_of_data):
    dir_name=str(name_dic[experiment])
    temp_var_name=type_of_data+'_'+dir_name
    return eval(temp_var_name)

In [5]:
concat_prol_meta=pd.concat([\
                           data('PANC1_proliferating_A3','meta'),\
                           data('PANC1_proliferating_B1','meta'),\
                           ],\
                           keys=[\
                           'PANC1_proliferating_A3',\
                           'PANC1_proliferating_B1',\
                           ])
#print(concat_prol_calc.shape)
concat_prol_calc=pd.concat([\
                           data('PANC1_proliferating_A3','calc'),\
                           data('PANC1_proliferating_B1','calc'),\
                           ],\
                           keys=[\
                           'PANC1_proliferating_A3',\
                           'PANC1_proliferating_B1',\
                           ])
concat_prol_calc=concat_prol_calc.rename(columns={'BlockId':'BlockId Calc'})
#print(concat_prol_calc.shape)
concat_prol_whole=pd.concat([concat_prol_meta,concat_prol_calc],axis=1)
concat_prol_whole=concat_prol_whole.drop(columns='BlockId Calc')

In [6]:
concat_prol_meta.shape

(496, 15)

In [7]:
m=concat_prol_whole.groupby('BlockId')['ExcessHSA'].mean().to_frame().reset_index().rename(columns={'ExcessHSA':'Mean ExcessHSA'})
s=concat_prol_whole.groupby('BlockId')['ExcessHSA'].std().to_frame().reset_index().rename(columns={'ExcessHSA':'STD ExcessHSA'}).drop(columns='BlockId')
prolA3B1_stat_HSA=pd.concat([m,s],axis=1)
prolA3B1_stat_HSA['RMSE over Mean ExcessHSA']=prolA3B1_stat_HSA['STD ExcessHSA']/prolA3B1_stat_HSA['Mean ExcessHSA']/2**0.5
######
m=concat_prol_whole.groupby('BlockId')['Beta'].mean().to_frame().reset_index().rename(columns={'Beta':'Mean Beta'}).drop(columns='BlockId')
s=concat_prol_whole.groupby('BlockId')['Beta'].std().to_frame().reset_index().rename(columns={'Beta':'STD Beta'}).drop(columns='BlockId')
prolA3B1_stat_Beta=pd.concat([m,s],axis=1)
prolA3B1_stat_Beta['RMSE over Mean Beta']=prolA3B1_stat_Beta['STD Beta']/prolA3B1_stat_Beta['Mean Beta']/2**0.5
######
m=concat_prol_whole.groupby('BlockId')['Gamma'].mean().to_frame().reset_index().rename(columns={'Gamma':'Mean Gamma'}).drop(columns='BlockId')
s=concat_prol_whole.groupby('BlockId')['Gamma'].std().to_frame().reset_index().rename(columns={'Gamma':'STD Gamma'}).drop(columns='BlockId')
prolA3B1_stat_Gamma=pd.concat([m,s],axis=1)
prolA3B1_stat_Gamma['RMSE over Mean Gamma']=prolA3B1_stat_Gamma['STD Gamma']/prolA3B1_stat_Gamma['Mean Gamma']/2**0.5
######
m=concat_prol_whole.groupby('BlockId')['DBSumNeg'].mean().to_frame().reset_index().rename(columns={'DBSumNeg':'Mean DBSumNeg'}).drop(columns='BlockId')
s=concat_prol_whole.groupby('BlockId')['DBSumNeg'].std().to_frame().reset_index().rename(columns={'DBSumNeg':'STD DBSumNeg'}).drop(columns='BlockId')
prolA3B1_stat_DBSumNeg=pd.concat([m,s],axis=1)
prolA3B1_stat_DBSumNeg['RMSE over Mean DBSumNeg']=prolA3B1_stat_DBSumNeg['STD DBSumNeg']/prolA3B1_stat_DBSumNeg['Mean DBSumNeg']/2**0.5
######
prolA3B1_stat=pd.concat([prolA3B1_stat_HSA,prolA3B1_stat_Beta,prolA3B1_stat_Gamma,prolA3B1_stat_DBSumNeg],axis=1)


In [8]:
#Read in all the data
for key in name_dic:    
    dir_name=str(name_dic[key]) #The data files for each experiment, i.e. key, is in a folder named name_dic[key].
    temp_var_name='meta_smiles_'+dir_name
    temp_dataframe=read_csv(dir_name+"/metadata_smiles.csv",header=0)
    exec(temp_var_name + " = temp_dataframe")

In [9]:
prolA3B1_stat['Label']=(prolA3B1_stat['Mean Gamma']<0.95).astype(int)
prolA3B1_stat['Label']=prolA3B1_stat['Label']

In [10]:
prolA3B1=pd.concat([data('PANC1_proliferating_A3','meta_smiles'),\
                    data('PANC1_proliferating_B1','meta_smiles')]).reset_index().drop(columns=['index','Unnamed: 15','Size','Row_resolved','Col_resolved','RowIC50','ColIC50'])
prolA3B1=pd.concat([prolA3B1,prolA3B1_stat['Label']],axis=1)

In [11]:
single_agents=read_csv("panc1_proliferate_single_agents.csv",header=0)
single_agents_split=[i.split('-')[0] for i in single_agents['Sample ID']]
single_agents['Sample ID split']=single_agents_split
single_agents_list=single_agents['Sample ID split'].tolist()

cond_cc=(single_agents['CC-v2']==-1.1) | (single_agents['CC-v2']==-1.2) | (single_agents['CC-v2']==-2.1) | (single_agents['CC-v2']==-2.2)
cond_ac50=single_agents['AC50 (uM)'] <= 30
single_agents_cc_1_2=single_agents[cond_cc & cond_ac50]
single_agents_cc_not_1_2=single_agents[~(cond_cc & cond_ac50)]

In [12]:
#Taking the average AC50s of row and column.
for indx, df_iterrows in prolA3B1.iterrows():
    ncgcid_row_split=df_iterrows['RowSid'].split('-')[0]
    ncgcid_col_split=df_iterrows['ColSid'].split('-')[0]
    cond_row=ncgcid_row_split==single_agents['Sample ID split']
    cond_col=ncgcid_col_split==single_agents['Sample ID split']
    if ncgcid_row_split=='NCGC00386226': #this compound is not in the list of single agents
        AC50row=0.053159193548387114 #This value was taken from the matrix screens
    else:
        AC50row=single_agents[cond_row]['AC50 (uM)'].values[0]
    if ncgcid_col_split=='NCGC00386226':
        AC50col=0.053159193548387114
    else:
        AC50col=single_agents[cond_col]['AC50 (uM)'].values[0]
    prolA3B1.at[indx,'AC50 Row (uM)']=AC50row
    prolA3B1.at[indx,'AC50 Col (uM)']=AC50col
    prolA3B1.at[indx,'Geometric Average of AC50s (uM)']=(AC50row*AC50col)**0.5
    prolA3B1['Log AC50 Row (M)']=np.log10( prolA3B1['AC50 Row (uM)']*(10**-6)  )
    prolA3B1['Log AC50 Col (M)']=np.log10( prolA3B1['AC50 Col (uM)']*(10**-6)  )
    prolA3B1['Arithmetic Average of Log AC50s (M)']=np.log10( prolA3B1['Geometric Average of AC50s (uM)']*(10**-6) )
 #   print(indx,ncgcid_row_split,ncgcid_col_split,AC50row,AC50col)

In [13]:
temp1=prolA3B1[['RowSid','Row SMILES','Log AC50 Row (M)','RowName','RowTarget','ColSid','Col SMILES','Log AC50 Col (M)','ColName','ColTarget']]
prolA3B1_stat['Label']=(prolA3B1_stat['Mean Gamma']<0.95).astype(int)
temp2=prolA3B1_stat[['Mean Gamma','Label']]
training_set=pd.concat([temp1,temp2],axis=1)
training_set.columns=['Sample ID 1','Structure 1','Log AC50 (M) 1','Sample Name 1','MOA 1','Sample ID 2','Structure 2','Log AC50 (M) 2','Sample Name 2','MOA 2','Gamma','Label']

In [14]:
training_set.shape

(496, 12)

In [15]:
training_set.isna().sum()

Sample ID 1       0
Structure 1       0
Log AC50 (M) 1    0
Sample Name 1     0
MOA 1             0
Sample ID 2       0
Structure 2       0
Log AC50 (M) 2    0
Sample Name 2     0
MOA 2             0
Gamma             0
Label             0
dtype: int64

# Preparing Test Set

In [474]:
# Step 1: Generate all combinations disregarding what has been tested so far
import datetime
print ('Began at = ',datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

size=single_agents.shape[0]
print('\nTotal number of single agents =',size)
chunk=0
for i in np.arange(0,size):
    if i%10 == 0:
        chunk = int( (i-i%10)/10   )
        temp_var_name='_'+str(chunk)
        exec(temp_var_name + " = np.empty(12)")
        print('\nChunk=',chunk,temp_var_name,end = '\n')
    print(i, end = ' ')
    for j in np.arange(i+1,size):
        temp1=np.concatenate((working.to_numpy()[i],working.to_numpy()[j]))
        exec(  temp_var_name+"=np.vstack(("+temp_var_name+",temp1))"    )
            
print ('\nEnded at = ',datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Began at =  2020-09-18 14:11:24

Total number of single agents = 1784

Chunk= 0 _0
0 1 2 3 4 5 6 7 8 9 
Chunk= 1 _1
10 11 12 13 14 15 16 17 18 19 
Chunk= 2 _2
20 21 22 23 24 25 26 27 28 29 
Chunk= 3 _3
30 31 32 33 34 35 36 37 38 39 
Chunk= 4 _4
40 41 42 43 44 45 46 47 48 49 
Chunk= 5 _5
50 51 52 53 54 55 56 57 58 59 
Chunk= 6 _6
60 61 62 63 64 65 66 67 68 69 
Chunk= 7 _7
70 71 72 73 74 75 76 77 78 79 
Chunk= 8 _8
80 81 82 83 84 85 86 87 88 89 
Chunk= 9 _9
90 91 92 93 94 95 96 97 98 99 
Chunk= 10 _10
100 101 102 103 104 105 106 107 108 109 
Chunk= 11 _11
110 111 112 113 114 115 116 117 118 119 
Chunk= 12 _12
120 121 122 123 124 125 126 127 128 129 
Chunk= 13 _13
130 131 132 133 134 135 136 137 138 139 
Chunk= 14 _14
140 141 142 143 144 145 146 147 148 149 
Chunk= 15 _15
150 151 152 153 154 155 156 157 158 159 
Chunk= 16 _16
160 161 162 163 164 165 166 167 168 169 
Chunk= 17 _17
170 171 172 173 174 175 176 177 178 179 
Chunk= 18 _18
180 181 182 183 184 185 186 187 188 189 
Chunk= 19 _19


1410 1411 1412 1413 1414 1415 1416 1417 1418 1419 
Chunk= 142 _142
1420 1421 1422 1423 1424 1425 1426 1427 1428 1429 
Chunk= 143 _143
1430 1431 1432 1433 1434 1435 1436 1437 1438 1439 
Chunk= 144 _144
1440 1441 1442 1443 1444 1445 1446 1447 1448 1449 
Chunk= 145 _145
1450 1451 1452 1453 1454 1455 1456 1457 1458 1459 
Chunk= 146 _146
1460 1461 1462 1463 1464 1465 1466 1467 1468 1469 
Chunk= 147 _147
1470 1471 1472 1473 1474 1475 1476 1477 1478 1479 
Chunk= 148 _148
1480 1481 1482 1483 1484 1485 1486 1487 1488 1489 
Chunk= 149 _149
1490 1491 1492 1493 1494 1495 1496 1497 1498 1499 
Chunk= 150 _150
1500 1501 1502 1503 1504 1505 1506 1507 1508 1509 
Chunk= 151 _151
1510 1511 1512 1513 1514 1515 1516 1517 1518 1519 
Chunk= 152 _152
1520 1521 1522 1523 1524 1525 1526 1527 1528 1529 
Chunk= 153 _153
1530 1531 1532 1533 1534 1535 1536 1537 1538 1539 
Chunk= 154 _154
1540 1541 1542 1543 1544 1545 1546 1547 1548 1549 
Chunk= 155 _155
1550 1551 1552 1553 1554 1555 1556 1557 1558 1559 
Chunk= 156 

In [531]:
test_df=pd.DataFrame()
for chunk in range(179):
    print(chunk,end=' ')
    temp_var_name='_'+str(chunk)
    exec(  'temp=pd.DataFrame(' + temp_var_name + ').drop([0])'   )
    test_df=pd.concat([test_df,temp])
test_df.columns=['Sample ID 1','Structure 1','Log AC50 (M) 1','Sample Name 1','MOA 1','Sample ID split 1','Sample ID 2','Structure 2','Log AC50 (M) 2','Sample Name 2','MOA 2','Sample ID split 2']
test_df.shape

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 

(1590436, 12)

In [543]:
test_df_prior_to_exclusion=test_df.reset_index().drop(columns=['index']).copy()
test_df_prior_to_exclusion.to_csv('Test_set_prior_to_exclusion.csv',sep=',',index=False)
print('Saved to Test_set_prior_to_exclusion.csv')
test_df_prior_to_exclusion=make_set_from_two_cols(test_df_prior_to_exclusion,'Sample ID split 1','Sample ID split 2','Combination Set')
combos_to_be_excluded=prolA3B1['Combination Set'].values
counter=0
for index,row in test_df_prior_to_exclusion.iterrows():
    #print(index,end=' ')
    if row['Combination Set'] in combos_to_be_excluded:
        print(row['Combination Set'])
        test_df_prior_to_exclusion=test_df_prior_to_exclusion.drop([index])
        counter=counter+1
print(counter)
test_df_after_exclusion=test_df_prior_to_exclusion.reset_index().drop(columns=['index','Combination Set']).copy()
print('\nTest set dimension = ',test_df_after_exclusion.shape)
test_df_after_exclusion.to_csv('Test_set.csv',sep=',',index=False)

Saved to Test_set_prior_to_exclusion.csv
{'NCGC00242514', 'NCGC00345782'}
{'NCGC00242514', 'NCGC00263017'}
{'NCGC00242514', 'NCGC00182868'}
{'NCGC00242514', 'NCGC00249613'}
{'NCGC00242514', 'NCGC00242506'}
{'NCGC00242514', 'NCGC00263110'}
{'NCGC00242514', 'NCGC00263087'}
{'NCGC00242514', 'NCGC00015038'}
{'NCGC00345799', 'NCGC00242514'}
{'NCGC00242514', 'NCGC00025060'}
{'NCGC00242514', 'NCGC00345823'}
{'NCGC00242514', 'NCGC00263117'}
{'NCGC00242514', 'NCGC00188382'}
{'NCGC00242514', 'NCGC00346962'}
{'NCGC00242514', 'NCGC00346951'}
{'NCGC00242514', 'NCGC00345825'}
{'NCGC00242514', 'NCGC00263183'}
{'NCGC00242514', 'NCGC00016268'}
{'NCGC00242514', 'NCGC00263222'}
{'NCGC00242514', 'NCGC00347952'}
{'NCGC00253438', 'NCGC00242514'}
{'NCGC00242514', 'NCGC00346644'}
{'NCGC00242514', 'NCGC00263112'}
{'NCGC00242514', 'NCGC00346656'}
{'NCGC00242514', 'NCGC00345809'}
{'NCGC00242514', 'NCGC00345852'}
{'NCGC00242514', 'NCGC00346449'}
{'NCGC00242514', 'NCGC00346535'}
{'NCGC00242514', 'NCGC00346969'}
{'

{'NCGC00345799', 'NCGC00346656'}
{'NCGC00345799', 'NCGC00345809'}
{'NCGC00345799', 'NCGC00345852'}
{'NCGC00345799', 'NCGC00346449'}
{'NCGC00345799', 'NCGC00346535'}
{'NCGC00345799', 'NCGC00346969'}
{'NCGC00345799', 'NCGC00242217'}
{'NCGC00345823', 'NCGC00025060'}
{'NCGC00263117', 'NCGC00025060'}
{'NCGC00188382', 'NCGC00025060'}
{'NCGC00346962', 'NCGC00025060'}
{'NCGC00346951', 'NCGC00025060'}
{'NCGC00345825', 'NCGC00025060'}
{'NCGC00263183', 'NCGC00025060'}
{'NCGC00016268', 'NCGC00025060'}
{'NCGC00263222', 'NCGC00025060'}
{'NCGC00347952', 'NCGC00025060'}
{'NCGC00253438', 'NCGC00025060'}
{'NCGC00346644', 'NCGC00025060'}
{'NCGC00263112', 'NCGC00025060'}
{'NCGC00346656', 'NCGC00025060'}
{'NCGC00345809', 'NCGC00025060'}
{'NCGC00345852', 'NCGC00025060'}
{'NCGC00346449', 'NCGC00025060'}
{'NCGC00346535', 'NCGC00025060'}
{'NCGC00346969', 'NCGC00025060'}
{'NCGC00242217', 'NCGC00025060'}
{'NCGC00263117', 'NCGC00345823'}
{'NCGC00188382', 'NCGC00345823'}
{'NCGC00346962', 'NCGC00345823'}
{'NCGC0034

In [599]:
test_df_after_exclusion=test_df_prior_to_exclusion.reset_index().drop(columns=['index','Combination Set','Sample ID split 1','Sample ID split 2']).copy()
print('\nTest set dimension = ',test_df_after_exclusion.shape)
test_df_after_exclusion.to_csv('Test_set.csv',sep=',',index=False)


Test set dimension =  (1589971, 10)


In [659]:
test_df_after_exclusion.shape

(1589971, 10)

### Adding NCGC00386226-04 to the list of combinations

In [689]:
cond1=prolA3B1['RowSid']=='NCGC00386226-04'
cond2=prolA3B1['ColSid']=='NCGC00386226-04'
NCGC00386226_single=prolA3B1[cond1 | cond2].loc[0].to_frame().T
NCGC00386226_single=NCGC00386226_single[['RowSid','Row SMILES','Log AC50 Row (M)','RowName','RowTarget','RowSid split']]
NCGC00386226_single.columns=['Sample ID 1','Structure 1','Log AC50 (M) 1','Sample Name 1','MOA 1','Sample ID split 1']
NCGC00386226_single

,Sample ID 1,Structure 1,Log AC50 (M) 1,Sample Name 1,MOA 1,Sample ID split 1
0,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226


In [692]:
size=single_agents.shape[0]
print('\nTotal number of single agents =',size)
NCGC00386226_combos=np.empty(12)
for i in np.arange(0,size):
    temp1=np.concatenate((NCGC00386226_single.to_numpy()[0],working.to_numpy()[i]))
    NCGC00386226_combos=np.vstack((NCGC00386226_combos,temp1))            
NCGC00386226_combos_df=pd.DataFrame(NCGC00386226_combos).drop([0])
NCGC00386226_combos_df.columns=['Sample ID 1','Structure 1','Log AC50 (M) 1','Sample Name 1','MOA 1','Sample ID split 1','Sample ID 2','Structure 2','Log AC50 (M) 2','Sample Name 2','MOA 2','Sample ID split 2']
make_set_from_two_cols(NCGC00386226_combos_df,'Sample ID split 1','Sample ID split 2','Combination Set')
#NCGC00386226_combos_df


Total number of single agents = 1784


,Sample ID 1,Structure 1,Log AC50 (M) 1,Sample Name 1,MOA 1,Sample ID split 1,Sample ID 2,Structure 2,Log AC50 (M) 2,Sample Name 2,MOA 2,Sample ID split 2,Combination Set
1,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00242514-01,COCC[N]1C(=[N+](CC2=NC=CN=C2)C3=C1C(=O)C4=CC=C...,-8.3816,Sepantronium bromide,Survivin Inhibitor,NCGC00242514,"{NCGC00386226, NCGC00242514}"
2,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00016447-02,C[C@@H]1O[C@@H](O[C@H]\2CC[C@]\3(C)[C@@H]4CC[C...,-8.3316,Proscillaridin,Cardiac glycoside,NCGC00016447,"{NCGC00386226, NCGC00016447}"
3,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00345782-01,ClC1=CC=C(OCCCCCCN\C(NC2=CC=NC=C2)=N/C#N)C=C1,-8.3316,GMX-1778,IKK beta Inhibitor,NCGC00345782,"{NCGC00386226, NCGC00345782}"
4,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00162409-03,CO[C@H]1\C=C/C=C(/C)C[C@H](C)[C@H](O)[C@H](C)\...,-8.0316,Bafilomycin A1,Vacuolar H+ ATPase (V-ATPase) Inhibitor,NCGC00162409,"{NCGC00386226, NCGC00162409}"
5,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00017394-08,C[C@@H]1O[C@@H](O[C@H]2C[C@@H](O)[C@]3(CO)[C@H...,-8.0316,G-Strophanthin,Na+/K+-ATPase Inhibitor,NCGC00017394,"{NCGC00386226, NCGC00017394}"
6,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00263017-01,COC1=C(OC)C=C2C(=NC(=NC2=C1)C3=CC=C(C=C3)N4CCN...,-7.7316,CPG-52364,"TLR 7, 8 and 9 Antagonist",NCGC00263017,"{NCGC00386226, NCGC00263017}"
7,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00163548-02,CS[C@H]1SC[C@H]2N(C)C(=O)[C@@H](C)NC(=O)[C@H](...,-7.6816,Echinomycin,Antineoplastic Antibiotics,NCGC00163548,"{NCGC00386226, NCGC00163548}"
8,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00182868-02,O=C(NCCCCC1CCN(CC1)C(=O)C2=CC=CC=C2)\C=C\C3=CN...,-7.5316,Daporinad,NAMPT Inhibitor,NCGC00182868,"{NCGC00386226, NCGC00182868}"
9,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00249613-03,CC(C)C[C@H](NC(=O)[C@H](CCC1=CC=CC=C1)NC(=O)CN...,-7.4816,Carfilzomib,Proteasome Inhibitor,NCGC00249613,"{NCGC00386226, NCGC00249613}"
10,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00386226,NCGC00242506-02,O=C(N[C@@H](Cc1ccccc1)C(=O)N[C@@H](CC(C)C)B(O)...,-7.4316,Bortezomib,Proteasome Inhibitor,NCGC00242506,"{NCGC00386226, NCGC00242506}"


In [693]:
NCGC00386226_combos_df=NCGC00386226_combos_df.reset_index().drop(columns=['index']).copy()
combos_to_be_excluded=prolA3B1['Combination Set'].values
counter=0
for index,row in NCGC00386226_combos_df.iterrows():
    #print(index,end=' ')
    if row['Combination Set'] in combos_to_be_excluded:
        print(row['Combination Set'])
        NCGC00386226_combos_df=NCGC00386226_combos_df.drop([index])
        counter=counter+1
print(counter)
NCGC00386226_combos_df=NCGC00386226_combos_df.reset_index().drop(columns=['index','Combination Set','Sample ID split 1','Sample ID split 2']).copy()
NCGC00386226_combos_df

{'NCGC00386226', 'NCGC00242514'}
{'NCGC00386226', 'NCGC00345782'}
{'NCGC00386226', 'NCGC00263017'}
{'NCGC00386226', 'NCGC00182868'}
{'NCGC00386226', 'NCGC00249613'}
{'NCGC00386226', 'NCGC00242506'}
{'NCGC00386226', 'NCGC00263110'}
{'NCGC00386226', 'NCGC00263087'}
{'NCGC00386226', 'NCGC00015038'}
{'NCGC00386226', 'NCGC00345799'}
{'NCGC00386226', 'NCGC00025060'}
{'NCGC00386226', 'NCGC00345823'}
{'NCGC00386226', 'NCGC00263117'}
{'NCGC00386226', 'NCGC00188382'}
{'NCGC00386226', 'NCGC00346962'}
{'NCGC00386226', 'NCGC00346951'}
{'NCGC00386226', 'NCGC00345825'}
{'NCGC00386226', 'NCGC00263183'}
{'NCGC00386226', 'NCGC00016268'}
{'NCGC00386226', 'NCGC00263222'}
{'NCGC00386226', 'NCGC00347952'}
{'NCGC00386226', 'NCGC00253438'}
{'NCGC00386226', 'NCGC00346644'}
{'NCGC00386226', 'NCGC00263112'}
{'NCGC00386226', 'NCGC00346656'}
{'NCGC00386226', 'NCGC00345809'}
{'NCGC00386226', 'NCGC00345852'}
{'NCGC00386226', 'NCGC00346449'}
{'NCGC00386226', 'NCGC00346535'}
{'NCGC00386226', 'NCGC00346969'}
{'NCGC0038

,Sample ID 1,Structure 1,Log AC50 (M) 1,Sample Name 1,MOA 1,Sample ID 2,Structure 2,Log AC50 (M) 2,Sample Name 2,MOA 2
0,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00016447-02,C[C@@H]1O[C@@H](O[C@H]\2CC[C@]\3(C)[C@@H]4CC[C...,-8.3316,Proscillaridin,Cardiac glycoside
1,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00162409-03,CO[C@H]1\C=C/C=C(/C)C[C@H](C)[C@H](O)[C@H](C)\...,-8.0316,Bafilomycin A1,Vacuolar H+ ATPase (V-ATPase) Inhibitor
2,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00017394-08,C[C@@H]1O[C@@H](O[C@H]2C[C@@H](O)[C@]3(CO)[C@H...,-8.0316,G-Strophanthin,Na+/K+-ATPase Inhibitor
3,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00163548-02,CS[C@H]1SC[C@H]2N(C)C(=O)[C@@H](C)NC(=O)[C@H](...,-7.6816,Echinomycin,Antineoplastic Antibiotics
4,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00346469-01,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)C1=CC=CC(=N1)C2...,-7.4316,Delanzomib,Proteasome Inhibitor
5,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00162381-03,[H][C@]12[C@H](OC(=O)CCCCCCC)[C@@H](OC(=O)C(\C...,-7.4316,Thapsigargin,SERCA Inhibitor
6,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00263220-02,C\C=C1/NC(=O)[C@H]2CSSCC\C=C/[C@H](CC(=O)N[C@H...,-7.3816,Romidepsin,HDAC Inhibitor
7,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00346692-01,COC1=CC=C(C=N1)C2=NC3=C(SC(=C3)CN(C)C4=NC=C(C=...,-7.3316,CUDC-907,PI3K Inhibitor
8,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00346550-01,CC1=C(N=C(NC(=O)C2(CC2)C3=CC4=C(OC(F)(F)O4)C=C...,-7.2316,Lumacaftor,CFTR Channel Modulator
9,NCGC00386226-04,NC1=NC(N)=C2N=C(CC(CC#C)C3=CC=C(C=C3)C(=O)N[C@...,-7.27442,Pralatrexate,Dihydrofolate Reductase (DHFR) Inhibitors,NCGC00346500-01,CN(/N=C/C1=CN=C2C=CC(=C[N]12)Br)[S+]([O-])(=O)...,-7.2316,PIK-75,PI3K Inhibitor


In [696]:
test_df_final=pd.concat([test_df_after_exclusion,NCGC00386226_combos_df])
print('The total number of combinations after adding NCGC00386226',test_df_final.shape)
test_df_final.to_csv('Test_set.csv',sep=',',index=False)

The total number of combinations after adding NCGC00386226 (1591724, 10)


In [700]:
1785*1784/2-496

1591724.0

In [679]:
a={'NCGC00242514','NCGC00263017'}
cond=prolA3B1['Combination Set']==a
prolA3B1[cond]

,AssayId,BlockId,RowSid,RowName,RowTarget,ColSid,ColName,ColTarget,RowConcs,ColConcs,RowConcUnit,ColConcUnit,Row SMILES,Col SMILES,Label,Row Mol Object,Col Mol Object,AC50 Row (uM),AC50 Col (uM),Geometric Average of AC50s (uM),Log AC50 Row (M),Log AC50 Col (M),Arithmetic Average of Log AC50s (M),RowSid split,ColSid split,Combination Set
89,9191,90,NCGC00263017-01,CPG-52364,"TLR 7, 8 and 9 Antagonist",NCGC00242514-13,Sepantronium bromide,Survivin Inhibitor,"20.0,10.0,5.0,2.5,1.25,0.625,0.3125,0.15625,0....","0.06,0.03,0.015,0.0075,0.00375,0.001875,9.3760...",uM,uM,COC1=C(OC)C=C2C(NCCN3CCOCC3)=NC(=NC2=C1)C4=CC=...,COCCN1C(C)=[N+](CC2=NC=CN=C2)C3=C1C(=O)C4=CC=C...,0,<rdkit.Chem.rdchem.Mol object at 0x0000021C01F...,<rdkit.Chem.rdchem.Mol object at 0x0000021C01F...,0.018552,0.004153,0.008778,-7.731609,-8.381638,-8.056624,NCGC00263017,NCGC00242514,"{NCGC00242514, NCGC00263017}"


In [16]:
test_set_final=pd.read_csv('Test_set.csv')

In [17]:
test_set_final.columns

Index(['Sample ID 1', 'Structure 1', 'Log AC50 (M) 1', 'Sample Name 1',
       'MOA 1', 'Sample ID 2', 'Structure 2', 'Log AC50 (M) 2',
       'Sample Name 2', 'MOA 2'],
      dtype='object')